In [ ]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist, FieldError
from archiv.models import *

In [ ]:
file = "archiv\data\SI_DB_Import.xlsx"

In [ ]:
df = pd.read_excel(file)[2:].fillna('False')

In [ ]:
failed_periods = []
failed_cadastralcommunities = []
for i, row in df.iterrows():
    if row['*Site ID'] != 'False':
        temp_site,_ = Site.objects.get_or_create(
            identifier=row['*Site ID']
        )
        if row['*Site Name'] != 'False':
            temp_site.name=row['*Site Name']
#         if row['Plot Number'] != 'False':
#             temp_site.plot_number=row['Plot Number']
        if row['Description\n'] != 'False':
            temp_site.description=row['Description\n']

    
    # ownership
    
        if row['Ownership'] != 'False':
            item = None
            item = row['Ownership']
            for x in SITE_OWNERSHIP:
                if x[0].endswith(item):
                    temp_site.ownership=x[0]
        
    
    # public
    
        if row['Public'] != 'False':
            if row['Public'] == 'yes':
                temp_site.public = True
    
    # altnames

        if row['Alternative Site Name'] != 'False':
            x = None
            for x in row['Alternative Site Name'].split(','):
                x = x.strip()
                temp_name, _ = AltName.objects.get_or_create(label=x)
                temp_site.alt_name.add(temp_name)
                temp_site.save()
                

    # related other_period
    
        if row['Other present periods'] != 'False':
            for y in row['Other present periods'].split(','):
                y = y.strip()
                try:
                    temp_period = SkosConcept.objects.get(pref_label__startswith=y)
                    temp_site.other_period.add(temp_period)
                except ObjectDoesNotExist:
                    failed_periods.append(y)
    
     # related places:
    
        if row['Cadastral Community'] != 'False':
            y = None
            for y in row['Cadastral Community'].split(','):
                y = y.strip()
                try:
                    temp_item = CadastralCommunity.objects.filter(cadcom_nam=y)
                    if len(temp_item) == 1:
                        temp_site.cadastral_community.add(temp_item[0])
                    else:
                        failed_cadastralcommunities.append("{} to many matches".format(y))
                except ObjectDoesNotExist:
                    print(y)
                    failed_cadastralcommunities.append(y)
        
        temp_site.save()


In [ ]:
# TOURISM
df = pd.read_excel(file, "TOURISM")[3:].fillna('False')

In [ ]:
temp_site = None
for i, row in df.iterrows():
    if row['*Site ID'] != 'False':
        temp_site = Site.objects.get(
            identifier=row['*Site ID']
        )
        if row['Accessibility'] != 'False':
            temp_site.accessibility = row['Accessibility']
        
        if row['Visibility'] != 'False':
            temp_site.visibility = row['Visibility']
        
        if row['Infrastructure'] != 'False':
            temp_site.infrastructure = row['Infrastructure']
        
        if row['Long-Term Management'] != 'False':
            temp_site.long_term_management = row['Long-Term Management']
        
        if row['Potential of the Surroundings'] != 'False':
            temp_site.potential_surrounding = row['Potential of the Surroundings']
        
        if row['App'] != 'False':
            if row['App'] == 'no':
                temp_site.iad_app = False
            else:
                temp_site.iad_app = True
        
        if row['AppDescription'] != 'False':
            temp_site.app_description = row['AppDescription']
        
        if row['Comment'] != 'False':
            temp_site.tourism_comment = row['Comment']
                
    
    
    temp_site.save()

In [ ]:
# Site.objects.all().delete()

In [ ]:
# ArchEnt.objects.all().delete()

In [ ]:
# MonumentProtection.objects.all().delete()

In [ ]:
# ResearchEvent.objects.all().delete()